# MoBioFP

## Import Python libraries

In [ ]:
import cv2
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path
from joblib import Memory

memory = Memory(location="cache/detection", verbose=0)

## Define global constants

In [ ]:
SAMPLE_DIR = "../data/raw/samples"
MODEL_CHECKPOINT = "YOLO_MODEL_CHECKPOINT"

In [ ]:
from mobiofp.utils import find_largest_connected_component, to_fingerprint, enhance_fingerprint

def show_images(images, titles, cmap="gray", show_axis=False, fig_size=10, sup_title=None):
    assert((titles is None) or (len(images) == len(titles)))

    num_images = len(images)
    num_cols = 4
    num_rows = math.ceil(num_images / num_cols)

    fig_height = fig_size * (num_rows / num_cols) * 1.5
    _, axes = plt.subplots(num_rows, num_cols, figsize=(fig_size, fig_height), constrained_layout=True)
    axes = axes.ravel()

    if sup_title:
        plt.suptitle(sup_title, fontsize=24)

    for idx, (image, title) in enumerate(zip(images, titles)):
        axes[idx].imshow(image, cmap=cmap)
        axes[idx].set_title(title, fontsize=12)
        axes[idx].axis("on" if show_axis else "off")
    
    # Hide the remaining subplots
    for idx in range(num_images, num_cols * num_rows):
        axes[idx].axis("off")

plt.show()

def show_iqa(sharpness_scores, contrast_scores, mask_coverage_scores):
    _, axes = plt.subplots(1, 3, figsize=(20, 5), constrained_layout=True)
    plt.suptitle("Fingertip Image-Quality Assesment", fontsize=16)

    sns.boxplot(sharpness_scores, ax=axes[0], color="blue")
    axes[0].set_xlabel("Sharpness Score")
    axes[0].set_ylabel("Density")

    sns.boxplot(contrast_scores, ax=axes[1], color="red")
    axes[1].set_xlabel("Contrast Score")
    axes[1].set_ylabel("Density")

    sns.boxplot(mask_coverage_scores, ax=axes[2], color="green")
    axes[2].set_xlabel("Binary Mask Coverage Score")
    axes[2].set_ylabel("Density")

    plt.show()

def post_process_mask(mask):
    # Apply morphological operation and Gaussian blur
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
    mask = cv2.GaussianBlur(mask, (5, 5), sigmaX=2, sigmaY=2, borderType=cv2.BORDER_DEFAULT)
    mask = np.where(mask < 127, 0, 255).astype(np.uint8)

    # Find the largest connected component
    mask = find_largest_connected_component(mask)

    return mask

def fingertip_enhancement(image):
    # Convert to grayscale
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Normalize, bilateral filter, and CLAHE
    image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
    image = cv2.bilateralFilter(image, 7, 50, 50)
    image = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)).apply(image)

    return image

@memory.cache
def from_fingertip_to_fingerprint(image):
    # Fingertip Adaptive Thresholding
    binary = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 2)

    try:
        fingerprint = to_fingerprint(binary)

        return fingerprint
    except Exception as e:
        print(f"Error converting fingerphoto to fingerprint: {e}")
        return None

@memory.cache
def fingerprint_enhancement(image):
    fingerprint = enhance_fingerprint(image)
    fingerprint = fingerprint.astype("uint8")

    return fingerprint

## Read sample images

In [ ]:
from imutils import rotate_bound

images_paths = list(Path(SAMPLE_DIR).rglob("*.jpg"))
images = []
images_titles = []

# Read sample images
for p in images_paths:
    img = cv2.imread(str(p))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = rotate_bound(img, 90)
    images.append(img)
    images_titles.append(p.stem)

show_images(images, images_titles, fig_size=15, sup_title="Sample Fingerphoto Images")

## Fingertip detection using YOLOv8n pre-trained model

In [ ]:
from ultralytics import YOLO

model = YOLO(MODEL_CHECKPOINT)
model.info()
results = model(images, stream=True, conf=0.80, max_det=1)

predicted_images = []
bbox_coords = []
fingertip_images = []

for result in results:
    boxes = result.boxes.xyxy.tolist()
    boxes = [int(coord) for coord in boxes[0]]
    bbox_coords.append(boxes)    
    
    original = result.orig_img
    x1, y1, x2, y2 = boxes
    fingertip = original[y1:y2, x1:x2]
    fingertip_images.append(fingertip)

    predicted = result.plot()
    predicted_images.append(predicted)

show_images(predicted_images, images_titles, fig_size=15, sup_title="YOLOv8n Fingertip Detection")

In [ ]:
show_images(fingertip_images, images_titles, sup_title="Fingertip Images")

In [ ]:
from rembg import remove, new_session

# Initialize the rembg session
rembg_session = new_session("isnet-general-use")

# Remove background from the cropped images
fingertip_masks = [remove(fingertip, only_mask=True, session=rembg_session) for fingertip in fingertip_images]

# Post-process masks
fingertip_masks = [post_process_mask(mask) for mask in fingertip_masks]

show_images(fingertip_masks, images_titles, sup_title="Fingertip Masks")

## Fingertip Image-Quality Assessment

In [ ]:
from mobiofp.utils import quality_scores

sharpness_scores = []
contrast_scores = []
mask_coverage_scores = []

for image, mask in zip(fingertip_images, fingertip_masks):
    sharpness_score, contrast_score, mask_coverage_scorere = quality_scores(image, mask)
    sharpness_scores.append(sharpness_score)
    contrast_scores.append(contrast_score)
    mask_coverage_scores.append(mask_coverage_scorere)

show_iqa(sharpness_scores, contrast_scores, mask_coverage_scores)

In [ ]:
print(f"Number of images before filtering: {len(fingertip_images)}")
print(f"Number of masks before filtering: {len(fingertip_masks)}")

# NOTE:
#
# For the scope of this demonstration, we will consider only the binary mask coverage score
# and we will not consider the sharpness and contrast scores. The binary mask coverage threshold
# is set to 70% only to include all the images in the dataset since the dataset is already too small
# and we want to demonstrate the next steps of the pipeline.
BINARY_MASK_COVERAGE_THRESH = 70.0

iqa_images = []
iqa_masks = []
iqa_titles = []

for mcs, fingertip, fingertip_mask, fingertip_title in zip(mask_coverage_scores, fingertip_images, fingertip_masks, images_titles):
    if mcs >= BINARY_MASK_COVERAGE_THRESH:
        iqa_images.append(fingertip)
        iqa_masks.append(fingertip_mask)
        iqa_titles.append(fingertip_title)

assert len(iqa_images) == len(iqa_masks) == len(iqa_titles)

print(f"Number of images after filtering: {len(iqa_images)}")
print(f"Number of masks after filtering: {len(iqa_masks)}")

## Fingertip Enhancement

In [ ]:
grayscale_images = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in iqa_images]
grayscale_images = [cv2.bitwise_and(image, image, mask=mask) for image, mask in zip(grayscale_images, iqa_masks)]

show_images(grayscale_images, iqa_titles, cmap="gray", sup_title="Grayscale Fingertip Images")

In [ ]:
fingertip_enhanced_images = [fingertip_enhancement(image) for image in grayscale_images]

show_images(fingertip_enhanced_images, iqa_titles, sup_title="Fingertip Enhanced Images")

## Contactless (fingertip fingerphoto) to Contact (fingerprint) Image Conversion

The function `to_fingerprint()` takes an fingertip-enhanced and converts it into a fingerprint image.
It does this by:

- Resizing the image.
- Calculating the local gradient of the image using Sobel filters.
- Calculating the orientation of the ridges in the fingerprint.
- Extracting a region of the image and smoothing it to reduce noise.
- Calculating the x-signature of the region and finding its local maxima to estimate the ridge period.
- Creating a bank of Gabor filters with different orientations.
- Filtering the image with each filter in the bank.
- Assembling the final result by taking the corresponding convolution result for each pixel based on the closest orientation in the Gabor bank.
- Converting the result to grayscale.

In [ ]:
fingerprints, fingerprint_titles = [], []

for image, title in zip(fingertip_enhanced_images, iqa_titles):
    fingerprint = from_fingertip_to_fingerprint(image)
    if fingerprint is not None:
        fingerprints.append(fingerprint)
        fingerprint_titles.append(title)

show_images(fingerprints, fingerprint_titles, sup_title="Fingerprint Images")

In [ ]:
# Apply fingerprint enhancement using Fingerprint-Enhancement-Python package
#
# Ref: https://github.com/Utkarsh-Deshmukh/Fingerprint-Enhancement-Python
#
# NOTE: This step is one of the slowest steps in the end-to-end pipeline for our fingerphoto matching algorithm
fingerprint_enhanced_images = [fingerprint_enhancement(fingerprint) for fingerprint in fingerprints]

show_images(fingerprint_enhanced_images, iqa_titles, sup_title="Fingerprint Enhanced Images")

## Save Contact (fingerprint) Images

In [ ]:
PROCESSED_DIR = Path("../data/processed/samples/detection")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

for image, title in zip(fingerprint_enhanced_images, fingerprint_titles):
    fingerprint_path = PROCESSED_DIR / f"{title}.png"
    cv2.imwrite(str(fingerprint_path), image)